In [1]:
# Cell 1: Import Libraries
import pandas as pd
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import torch
from tqdm.auto import tqdm # For a progress bar

print("Libraries imported successfully.")

Libraries imported successfully.


In [2]:
# Cell 2: Initialize Models and Pinecone
# --- IMPORTANT ---
# Replace with your actual Pinecone API Key and Environment
PINECONE_API_KEY = "pcsk_4vgmFt_4ABagzHLAoEzjBmZX1VoEPZZDGJUZ6wNnoiNDkRCT6s32h6fRQyWpSH35LJGqdu"
PINECONE_ENVIRONMENT = "us-east-1"
PINECONE_INDEX_NAME = "product-recommendations"

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Connect to the index
index = pc.Index(PINECONE_INDEX_NAME)

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the multi-modal CLIP model
# This model can embed both text and images into the same vector space.
# --- UPDATED CODE ---
# Define the path to the folder where you saved the model
# The '../' means 'go up one directory' from our current 'notebooks' folder
local_model_path = "../local_models/clip-ViT-B-32"

# Load the model from the local directory
model = SentenceTransformer(local_model_path, device=device)

print("CLIP model loaded successfully.")

Using device: cpu


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


CLIP model loaded successfully.


In [3]:
# Cell 3: Load the Cleaned Dataset
# We use the cleaned dataset we saved in the previous notebook.
df = pd.read_csv('cleaned_dataset.csv')

# Let's fill any remaining NaNs in object columns just in case
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].fillna('N/A')

print("Cleaned dataset loaded.")
df.head()

Cleaned dataset loaded.


,title,brand,description,price,categories,images,manufacturer,package_dimensions,country_of_origin,material,color,uniq_id
0,"GOYMFK 1pc Free Standing Shoe Rack, Multi-laye...",GOYMFK,"multiple shoes, coats, hats, and other items E...",24.99,"['Home & Kitchen', 'Storage & Organization', '...",['https://m.media-amazon.com/images/I/416WaLx1...,GOYMFK,"2.36""D x 7.87""W x 21.6""H",China,Metal,White,02593e81-5c09-5069-8516-b0b29f439ded
1,Plant Repotting Mat MUYETOL Waterproof Transpl...,MUYETOL,N/A,5.98,"['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...",['https://m.media-amazon.com/images/I/41RgefVq...,MUYETOL,"26.8""L x 26.8""W",N/A,Polyethylene,Green,b2ede786-3f51-5a45-9a5b-bcf856958cd8
2,"Pickleball Doormat, Welcome Doormat Absorbent ...",VEWETOL,The decorative doormat features a subtle textu...,13.99,"['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...",['https://m.media-amazon.com/images/I/61vz1Igl...,Contrence,"24""L x 16""W",N/A,Rubber,A5589,8fd9377b-cfa6-5f10-835c-6b8eca2816b5
3,JOIN IRON Foldable TV Trays for Eating Set of ...,JOIN IRON Store,Set of Four Folding Trays With Matching Storag...,89.99,"['Home & Kitchen', 'Furniture', 'Game & Recrea...",['https://m.media-amazon.com/images/I/41p4d4VJ...,N/A,"18.9""D x 14.2""W x 26""H",N/A,Iron,Grey Set of 4,bdc9aa30-9439-50dc-8e89-213ea211d66a
4,Folews Bathroom Organizer Over The Toilet Stor...,Folews Store,N/A,63.99,"['Home & Kitchen', 'Furniture', 'Bathroom Furn...",['https://m.media-amazon.com/images/I/41ixgM73...,Folews,"12.6""D x 25.2""W x 68.5""H",China,N/A,N/A,aba4138e-6401-52ca-a099-02e30b638db4


In [5]:
# Cell 4: Prepare Data for Upsert
# Reasoning: We create a 'combined_text' field to generate a rich embedding.
# The metadata will store crucial product information that we want to retrieve
# along with the vector during a search query.

df['combined_text'] = (
    "Title: " + df['title'] + ". " +
    "Brand: " + df['brand'] + ". " +
    "Description: " + df['description'].str.slice(0, 200) + ". " + # Truncate long descriptions
    "Material: " + df['material'] + ". " +
    "Color: " + df['color'] + "."
)

# First, let's see how many rows we actually have
print(f"Total rows in the cleaned dataframe: {len(df)}")

# Now, sample the smaller of 2000 or the total length of the dataframe
sample_size = min(2000, len(df))
df_sample = df.sample(n=sample_size, random_state=42)

print(f"Processing a sample of {len(df_sample)} products.")

Total rows in the cleaned dataframe: 215
Processing a sample of 215 products.


In [6]:
# Cell 5: Generate Embeddings and Upsert to Pinecone in Batches
# Reasoning: Upserting in batches is much more efficient than one by one.

batch_size = 64
for i in tqdm(range(0, len(df_sample), batch_size)):
    # Get the current batch
    i_end = min(i + batch_size, len(df_sample))
    batch = df_sample.iloc[i:i_end]

    # Get unique IDs for the batch
    ids = [str(x) for x in batch['uniq_id']]

    # Create text embeddings
    texts = batch['combined_text'].tolist()
    text_embeddings = model.encode(texts).tolist()

    # Prepare metadata
    metadata = []
    for idx, row in batch.iterrows():
        meta = {
            'title': row['title'],
            'brand': row['brand'],
            'price': row['price'],
            'description': row['description'],
            'image_url': row['images'].split(',')[0] if isinstance(row['images'], str) else '', # Get first image URL
            'material': row['material'],
            'color': row['color']
        }
        metadata.append(meta)

    # Prepare records for upsert
    records_to_upsert = zip(ids, text_embeddings, metadata)

    # Upsert to Pinecone
    index.upsert(vectors=list(records_to_upsert))

print("\nUpsert to Pinecone complete.")
print(f"Pinecone index stats: {index.describe_index_stats()}")

  0%|          | 0/4 [00:00<?, ?it/s]


Upsert to Pinecone complete.
Pinecone index stats: {'dimension': 512,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 210}},
 'total_vector_count': 210,
 'vector_type': 'dense'}
